# *Введение в машиннное обучение. Проект*

# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [20]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [21]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

Разобьем выборку на обучающую, валидационную и тестовую. Для этого с помощью метода *train_test_split* выделим 60% для обучающей выборки. Оставшиеся 40% разделим пополам для валидационной и тестовой выборки.

In [23]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_temp, target_train, target_temp = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_temp, target_temp, test_size=0.5, random_state=12345)

In [24]:
print('Размер выборки _train', features_train.shape)
print('Размер выборки _valid', features_valid.shape)
print('Размер выборки _test', features_test.shape)

Размер выборки _train (1928, 4)
Размер выборки _valid (643, 4)
Размер выборки _test (643, 4)


## Исследуйте модели

Для выбора лучшей модели выберем лучшие гиперпараметры решающего дерева **DecisionTreeClassifier** и случайоного леса **RandomForestClassifier**

In [25]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions) 
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        
print("Accuracy лучшей модели DT на валидационной выборке:", best_result)
print("Depth лучшей модели:", best_depth)

Accuracy лучшей модели DT на валидационной выборке: 0.7853810264385692
Depth лучшей модели: 3


Для **RandomForestClassifier** будем использовать следующие параметры:

- n_estimators – число деревьев в лесу. Оно будет изменяться от 1 до 10 с шагом 2
- max_depth – глубина дерева. Она будет изменяться от 1 до 12 с шагом в 2
- min_samples_leaf – минимальное число образцов в листах. Оно будет изменяться от 1 до 7
- min_samples_leaf – минимальное число образцов для сплита. Оно будет изменяться от 2 до 9.

Для подбора параметров используем **GridSearchCV**, который находит наилучшие параметры путем обычного перебора: он создает модель для каждой возможной комбинации параметров. 

In [26]:
from sklearn.model_selection import GridSearchCV

model_rf = RandomForestClassifier() 

parametrs = { 'n_estimators': range (1, 11, 2),
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }
grid = GridSearchCV(model_rf, parametrs, cv=5)
grid.fit(features_train, target_train)
grid.best_params_

{'max_depth': 9,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 7}

In [30]:
model_best = RandomForestClassifier(
    random_state=12345, max_depth = 9, min_samples_leaf = 2, min_samples_split = 8, n_estimators = 7)

model_best.fit(features_train, target_train) 
predictions = model_best.predict(features_valid)
result = accuracy_score(target_valid, predictions)

print("Accuracy лучшей модели RF на валидационной выборке:", result)

Accuracy лучшей модели RF на валидационной выборке: 0.7916018662519441


**Вывод:** Лучшая модель - случайный лес с параметрами:
- 'max_depth': 9,
- 'min_samples_leaf': 2,
- 'min_samples_split': 8,
- 'n_estimators': 7

## Проверьте модель на тестовой выборке

In [31]:
model = RandomForestClassifier(
    random_state=12345, max_depth = 9, min_samples_leaf = 2, min_samples_split = 8, n_estimators = 7)
model.fit(features_train, target_train) 
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)

print("Accuracy на тестовой выборке:", result)

Accuracy на тестовой выборке: 0.8133748055987559


## (бонус) Проверьте модели на адекватность

Для проверки адекватности полученной модели используем **DummyClassifier**. Он делает прогнозы, которые игнорируют входные функции.
Этот классификатор служит простой базой для сравнения с другими более сложными классификаторами.

In [32]:
dummy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)

print('Accuracy dummy-модели: ', accuracy_score(target_test, dummy_pred))

Accuracy dummy-модели:  0.6842923794712286


# Вывод

Нами построена модель машинного обучения с помощью **RandomForestClassifier**, *accuracy* которой составила 0.81. Модель прошла проверку на адекватность, поскольку *accuracy* dummy-модели (которая не учитываем входные признаков) всего 0.68